In [68]:
import numpy as np
import cv2
import glob
from skvideo.io import vreader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#Length of the extracted feature
FEATURE_LENGTH = 3
#How many frames to sample from a video
SAMPLE_NUM = 30 
#The method for frame sampling from a video
SAMPLE_METHOD = "top_K"

In [69]:
def get_video_length(video_path):
    cap = cv2.VideoCapture(video_path)
    i = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break
        i +=1 
    return i


def get_video_paths(target_dir):
    video_paths = glob.glob(target_dir + "/*/*.mp4")
    return video_paths

def extract_frames(video_path, samples=3, method="top_K"):
    
    frames = []
    if method == "top_K":
        cap = cv2.VideoCapture(video_path)
        while(cap.isOpened()):
            ret, frame = cap.read()
            frames.append(frame)
            if len(frames) >= samples:
                return frames

            
        
    elif method == "random_K":
        video_length = get_video_length(video_path)
        sample_index = np.random.choice(video_length, samples)

        cap = cv2.VideoCapture(video_path)
        i = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if frame is None:
                break
            elif i in sample_index:
                frames.append(frame)

            i +=1 
            
    return frames


def extract_feature(img):
    frame = img
    roiWid = 10
    roiEdg = 8
    src_height, src_width, src_channels = frame.shape
    roiX = int(src_width / roiWid)
    roiWidth = roiX * roiEdg
    roiY = int(src_height / roiWid)
    roiHeight = roiY * roiEdg
    RoiImg = frame[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
 
    
    HSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    H,S,V = cv2.split(HSV)
    Vue_STD = np.std(V)
    
    r = img[...,2]
    g = img[...,1]
    b = img[...,0]
    rY = 0.212655;
    gY = 0.715158;
    bY = 0.072187;
    inner = RoiImg
    ir = inner[...,2]
    ig = inner[...,1]
    ib = inner[...,0]

    Luminance = r*0.2126+g*0.7152+0.0722*b
    Li = ir*0.2126+ig*0.7152+0.0722*ib
    hei,wid,chan = img.shape
    ih,iw,ic = inner.shape
    Lum_value = (np.sum(Luminance)-np.sum(Li))/((hei*wid*255)-(ih*iw*255))
    
    mr = np.sqrt((np.mean(r)-119)*(np.mean(r)-119))
    mg = np.sqrt((np.mean(g)-119)*(np.mean(g)-119))
    mb = np.sqrt((np.mean(b)-119)*(np.mean(b)-119))
    GW_value = (mr+mg+mb)/(3*255)
    
    return (Vue_STD,Lum_value,GW_value)

In [70]:
input_dirs = {"light":"Light", "dark":"Dark"}
frames = []
Y = []
for luminance, input_dir in input_dirs.items():
    is_dark = True if luminance == "dark" else False   
    video_paths = get_video_paths(input_dir)
    for video_path in video_paths:
        new_frame = extract_frames(video_path, SAMPLE_NUM, SAMPLE_METHOD)
        frames += new_frame
        Y += ([is_dark] * len(new_frame))
        
        
#Data Standardization for X and Y
X = np.zeros((len(frames), FEATURE_LENGTH))
i = 0
for i, frame in enumerate(frames):
    X[i] = extract_feature(frame)
Y = np.array(Y, dtype=bool)
        
        

In [71]:
#10-Fold Cross Validation
for i in range(10):
    scores = []
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=i)
    classifier = LogisticRegression()
    classifier.fit(X_train, Y_train)
    scores.append(classifier.score(X_test, Y_test))
print(sum(scores) / len(scores))

0.671497584541
